In [ ]:
import glob, pylab, pandas as pd
import pydicom, numpy as np
import cv2
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import os
import sys
import scipy.io
from PIL import Image
import numpy as np
import tensorflow as tf
import imageio
import h5py
import tables
from decimal import Decimal
import time
import functools
from functools import reduce
%matplotlib inline

In [ ]:
df = pd.read_csv('../data/stage_1_train_labels.csv')
df_class = pd.read_csv('../data/stage_1_detailed_class_info.csv')

In [ ]:
all_img_ids = []
for j in range(df.shape[0]):
    all_img_ids.append(df['patientId'][j])

In [ ]:
all_img_ids = shuffle(all_img_ids, random_state = 1)

In [ ]:
def parse_data(df):
    """
    Method to read a CSV file (Pandas dataframe) and parse the 
    data into the following nested dictionary:

      parsed = {
        
        'patientId-00': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        },
        'patientId-01': {
            'dicom': path/to/dicom/file,
            'label': either 0 or 1 for normal or pnuemonia, 
            'boxes': list of box(es)
        }, ...

      }

    """
    # --- Define lambda to extract coords in list [y, x, height, width]
    extract_box = lambda row: [row['y'], row['x'], row['height'], row['width']]

    parsed = {}
    for n, row in df.iterrows():
        # --- Initialize patient entry into parsed 
        pid = row['patientId']
        if pid not in parsed:
            parsed[pid] = {
                'dicom': '../data/stage_1_train_images/%s.dcm' % pid,
                'label': row['Target'],
                'boxes': []}

        # --- Add box if opacity is present
        if parsed[pid]['label'] == 1:
            parsed[pid]['boxes'].append(extract_box(row))

    return parsed

In [ ]:
parsed = parse_data(df)

In [ ]:
def draw(data):
    """
    Method to draw single patient with bounding box(es) if present 

    """
    # --- Open DICOM file
    d = pydicom.read_file(data['dicom'])
    im = d.pixel_array

    # --- Convert from single-channel grayscale to 3-channel RGB
    im = np.stack([im] * 3, axis=2)

    # --- Add boxes with random color if present
    for box in data['boxes']:
        rgb = [255,0,0]#np.floor(np.random.rand(3) * 256).astype('int')
        im = overlay_box(im=im, box=box, rgb=rgb, stroke=6)

    pylab.imshow(im,cmap=pylab.cm.gist_gray)

def overlay_box(im, box, rgb, stroke=1):
    """
    Method to overlay single box on image

    """
    # --- Convert coordinates to integers
    box = [int(b) for b in box]
    
    # --- Extract coordinates
    y1, x1, height, width = box
    y2 = y1 + height
    x2 = x1 + width

    im[y1:y1 + stroke, x1:x2] = rgb
    im[y2:y2 + stroke, x1:x2] = rgb
    im[y1:y2, x1:x1 + stroke] = rgb
    im[y1:y2, x2:x2 + stroke] = rgb

    return im

In [ ]:
#this ensures the program can use all the gpu resources it can get
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
# initialize weights
def weight_initializer(weight_input, output_channel_size, filter_size): #, layer_num
    
    _, rows, columns, input_channel_size = [i.value for i in weight_input.get_shape()]
    
    weight_shape = [filter_size,filter_size,input_channel_size,output_channel_size]

    weight_output = tf.Variable(tf.contrib.layers.xavier_initializer(uniform = False)(weight_shape))
    
    #weight_output = tf.get_variable(shape = weight_shape, dtype=tf.float32, 
                                    #initializer = tf.contrib.layers.xavier_initializer(uniform = False)) 
    #name = "weight-" + str(layer_num),
    
    return weight_output

In [ ]:
# convolution block 
def conv2d(block_input, num_filters, filter_size = 1, stride_length = 1): #, layer_num
    
    init_weights = weight_initializer(block_input, num_filters, filter_size) #, layer_num
    strides = [1,stride_length,stride_length,1]
    block_output = tf.nn.conv2d(block_input,init_weights,strides,padding='VALID')
    
    return block_output

In [ ]:
def conv_bn_relu(block_input, num_filters, filter_size = 1, stride_length = 1): #, layer_num
    
    init_weights = weight_initializer(block_input, num_filters, filter_size) #, layer_num
    strides = [1,stride_length,stride_length,1]
    
    block_output = tf.nn.conv2d(block_input,init_weights,strides,padding='VALID')
    normalized = tf.contrib.layers.batch_norm(block_output, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    
    return normalized

In [ ]:
def conv_block(block_input, num_filters):
    norm_1 = tf.contrib.layers.batch_norm(block_input, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    conv_1 = conv2d(norm_1, int(num_filters/2), 1, 1)
    norm_2 = tf.contrib.layers.batch_norm(conv_1, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    pad = tf.pad(norm_2, np.array([[0,0],[1,1],[1,1],[0,0]]))
    conv_2 = conv2d(pad, int(num_filters/2), 3, 1)
    norm_3 = tf.contrib.layers.batch_norm(conv_2, 0.9, epsilon=1e-5, activation_fn = tf.nn.relu)
    conv_3 = conv2d(norm_3, int(num_filters), 1, 1)
    
    return conv_3

In [ ]:
def skip_layer(block_input, num_filters):
    
    if (block_input.get_shape()[3] == num_filters):
        return block_input
    else:
        conv = conv2d(block_input, num_filters,1,1)
        return conv

In [ ]:
def residual(block_input, num_filters):
    conv = conv_block(block_input, num_filters)
    skip = skip_layer(block_input, num_filters)
    
    return(tf.add_n([conv,skip]))

In [ ]:
def hourglass_unit(input_data, reduction_factor, num_filters):
    up_1 = residual(input_data, num_filters)
    low = tf.contrib.layers.max_pool2d(input_data, [2,2],[2,2], 'VALID')
    low_1 = residual(low, num_filters)
    
    if reduction_factor > 0:
        low_2 = hourglass_unit(low_1, reduction_factor - 1, num_filters)
    else:
        low_2 = residual(low_1, num_filters)
    
    low_3 = residual(low_2, num_filters)
    up_sample = tf.image.resize_nearest_neighbor(low_3, tf.shape(low_3)[1:3]*2)
    return tf.add_n([up_1, up_sample])

In [ ]:
def hourglass_model(input_data, num_blocks, num_filters, reduction_factor, train_model):
    pad_1 = tf.pad(input_data, np.array([[0,0],[2,2],[2,2],[0,0]]))
    conv_1 = conv2d(pad_1, 64,6,2)
    res_1 = residual(conv_1, 128)
    pool_1 = tf.contrib.layers.max_pool2d(res_1, [2,2], [2,2], padding= 'VALID')
    res_2 = residual(pool_1, 128)
    res_3 = residual(res_2, num_filters)
    
    x1 = [None] * num_blocks
    x2 = [None] * num_blocks
    x3 = [None] * num_blocks
    x4 = [None] * num_blocks
    x5 = [None] * num_blocks
    x6 = [None] * num_blocks
    sum_all = [None] * num_blocks
    
    x1[0] = hourglass_unit(res_3, reduction_factor, num_filters)
    x2[0] = conv_bn_relu(x1[0], num_filters)
    x3[0] = conv2d(x2[0], num_filters, 1, 1)
    x4[0] = tf.layers.dropout(x3[0], rate = 0.1, training = train_model)
    x5[0] = conv2d(x2[0], num_filters, 1, 1)
    x6[0] = conv2d(x5[0], num_filters, 1, 1)
    sum_all[0] = tf.add_n([x4[0], x6[0], res_3])
    
    for i in range(1, num_blocks - 1):
        x1[i] = hourglass_unit(sum_all[i-1], reduction_factor, num_filters)
        x2[i] = conv_bn_relu(x1[i], num_filters)
        x3[i] = conv2d(x2[i], num_filters, 1, 1)
        x4[i] = tf.layers.dropout(x3[i], rate = 0.1, training = train_model)
        x5[i] = conv2d(x2[i], num_filters, 1, 1)
        x6[i] = conv2d(x5[i], num_filters, 1, 1)
        sum_all[i] = tf.add_n([x4[i], x6[i], sum_all[i-1]])
    
    x1[num_blocks - 1] = hourglass_unit(sum_all[num_blocks - 2], reduction_factor, num_filters)
    x2[num_blocks - 1] = conv_bn_relu(x1[num_blocks - 1], num_filters)
    x4[num_blocks - 1] = tf.layers.dropout(x2[num_blocks - 1], rate = 0.1, training = train_model)
    x5[num_blocks - 1] = conv2d(x4[num_blocks - 1], 3, 1, 1)
    final_output = tf.image.resize_nearest_neighbor(x5[num_blocks - 1], tf.shape(x5[num_blocks - 1])[1:3]*2)
    #final_output = x5[num_blocks - 1]
    return final_output

In [ ]:
batch_size = 4
number_train_images = df.shape[0]

In [ ]:
def get_lung_imgs(index1, index2, aug):
    
    if aug:
        X_batch = np.zeros((batch_size*4,256,256,3), dtype=np.float32)
    else:
        X_batch = np.zeros((batch_size,256,256,3), dtype=np.float32)
        
    k = 0
    for index in range(index1, index2, 1):
        d = pydicom.read_file(parsed[all_img_ids[index]]['dicom'])
        im = d.pixel_array
        im = np.stack([im] * 3, axis=2)
        im = cv2.resize(im, (256,256), interpolation = cv2.INTER_AREA)
        X_batch[k] = im
        k = k + 1
        
        if aug:
            rot_mat = cv2.getRotationMatrix2D((128,128), 30, 1.0)
            rot30 = cv2.warpAffine(im, rot_mat, (256,256))
            X_batch[k] = rot30
            k = k + 1

            rot_mat = cv2.getRotationMatrix2D((128,128), -30, 1.0)
            rot30 = cv2.warpAffine(im, rot_mat, (256,256))
            X_batch[k] = rot30        
            k = k + 1

            blurred = cv2.GaussianBlur(im,(5,5),cv2.BORDER_DEFAULT)
            X_batch[k] = blurred        
            k = k + 1
    
    return X_batch

In [ ]:
def get_marked_imgs(index1, index2, aug):
    size = 128
    
    if aug:
        Y_batch = np.zeros((batch_size*4,size,size,3), dtype=np.float32)
    else:
        Y_batch = np.zeros((batch_size,size,size,3), dtype=np.float32)
        
    k = 0
    for index in range(index1, index2, 1):
        img_black = np.zeros((size,size,3), np.float32)
        
        x = str(df_class.loc[df_class['patientId'] == all_img_ids[index]]['class'])
        
        if(x.split(' ')[4] == 'No'):
            cv2.rectangle(img_black, (25,25), (50,85), (0,40,0),-1)
            cv2.rectangle(img_black, (75,25), (100,85), (0,40,0),-1)
        
        
        for box in parsed[all_img_ids[index]]['boxes']:
            y = box[0]
            x = box[1]
            height = box[2]
            width = box[3]
            x1 = x*size/1024.0
            y1 = y*size/1024.0
            x2 = (x + width)*size/1024.0
            y2 = (y + height)*size/1024.0
            cv2.rectangle(img_black, (int(x1),int(y1)), (int(x2),int(y2)), (255,0,0),-1)   
            
            
        Y_batch[k] = img_black #normal img
        k = k + 1

        if aug:
            rot_mat = cv2.getRotationMatrix2D((64,64), 30, 1.0)
            rot30 = cv2.warpAffine(img_black, rot_mat, (128,128))
            Y_batch[k] = rot30
            k = k + 1

            rot_mat = cv2.getRotationMatrix2D((64,64), -30, 1.0)
            rot30 = cv2.warpAffine(img_black, rot_mat, (128,128))
            Y_batch[k] = rot30
            k = k + 1

            Y_batch[k] = img_black #blurred image
            k = k + 1
    
    return Y_batch

In [ ]:
with tf.Graph().as_default(),tf.Session() as sess:
    
    #for lungs_4_2 batch_size remains same. for lungs_3_5 -> batch_size*4 due to augmentations.
    lung_x_input = tf.placeholder(tf.float32,shape=(batch_size*4,256,256,3),name='lung_x_ip')
    lung_y_input = tf.placeholder(tf.float32,shape=(batch_size*4,128,128,3),name='lung_y_ip')
    ###################################################################################################################
    
    lung_x_input = lung_x_input/255.0
    
    hg_output = hourglass_model(lung_x_input, 4, 256, 3, True) # true while training, false during inference
    ###################################################################################################################
    
    #calculate mse losses 
    lung_y_input = lung_y_input/255.0
    total_loss = tf.losses.mean_squared_error(lung_y_input, hg_output)
    
    ###################################################################################################################
    #20180905 rate = 2.5e-4. 740 - 648-444-300
    #2018105 same rate loss 300-288
    #lungs_4_2 loss upto 270, lungs_3_5 loss up to 250
    #lungs_4_2 with green masks and red masks no aug
    #lungs_3_5 green and red masks with aug
    learning_rate = 2.5e-4
    run = 6
    training_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)
    num_epochs = 3
    sess.run(tf.global_variables_initializer())
    
    ###################################################################################################################
    
    restore_model = True
    save_model = True
    train_data = True
    
    #restore variable values. while saving the model further below, im only saving variable values and not the graph. 
    if(restore_model):
        saver =  tf.train.Saver()  
        saver.restore(sess,'../models/20181015/lungs_3_5')
    ###################################################################################################################
    
    if train_data:   
        
        num_minibatch = int(number_train_images/batch_size)
        
        for i in range(num_epochs+1):
            print("epoch number is ", i)
            batch_loss = 0.0

            start_time = time.time()

            for j in range(num_minibatch):
                #all img ids have all ids of img in shuffled way
                temp_x_content = get_lung_imgs(j*batch_size,batch_size*(j+1), True)
                temp_y_content = get_marked_imgs(j*batch_size,batch_size*(j+1), True)

                _,tl = sess.run([training_step,total_loss], feed_dict={lung_x_input:temp_x_content, 
                                                                       lung_y_input:temp_y_content})
                batch_loss += tl/num_minibatch

            end_time = time.time()
            print("total loss is ", batch_loss)
            print("epoch time is ", (end_time - start_time))

            #save the model without the graph.
            if(save_model and i%1 == 0):
                saver_2 = tf.train.Saver()  
                saver_2.save(sess,"../models/20181015/lungs_" + str(i) + "_" + str(run),write_meta_graph=False) 
    ###################################################################################################################